Outline
    
1. Import datasets

2. Data cleaning + export cleaned datasets to `datasets/cleaned` (Note: index column name should always be "airport_code")

    2.1 Clean `datasets/original/crash` (unfinished data1)
    
    2.2 Clean `datasets/original/delay/2009.csv`. Export `unpleasant_2009.csv`.
    
    2.3 Clean `datasets/original/airport/airport-extended.csv`. Generate `airport_loc_df` with column names city_name, latitude, longitude, altitude_ft.
    
    2.4 Clean `datasets/original/city/uscities.csv`.
    
    2.5 Merge `unpleasant_2009` and `airport_loc_df` to get pre-`airport_prop_df`. Drop rows whose city_name isn't included in `us_cities_df`. Scrape pcp data with datasets in `datasets/original/weather`. Then we get `airport_prop_df`. Export `airport_prop_df.csv`.
    
    2.6 Clean `datasets/original/airports/Bird Strikes.xlsx`. (unfinished)
    
    
    

Note: 2.1 ~ 2.5 are from `A Pleasant Flight.ipynb`

==========================================

In [1]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
import matplotlib.pyplot as plt
import requests
from string import digits
import wget # you need to "pip install wget"  
import glob
import time

import plotly.graph_objects as go # use conda to install plotly --YD
import pandas as pd
import plotly.figure_factory as ff
import xml.etree.ElementTree as ET # to read one dataset in XML format

1. import datasets

From `A Pleasant Flight.ipynb` (now in the folder `previous_codes`)

In [2]:
crash_df = pd.read_csv("datasets/original/crash/Airplane_Crashes_and_Fatalities_Since_1908.csv") #data1
etree = ET.parse("datasets/original/crash/AviationData.xml") #data2
delay_2009_df = pd.read_csv("datasets/original/delay/2009.csv") #data3 (done)
    #need to download 2009.csv from https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

In [3]:
airport_loc_df = pd.read_csv("datasets/original/airport/airports-extended.csv") #data4
#https://www.kaggle.com/open-flights/airports-train-stations-and-ferry-terminals#airports-extended.csv

#aiport_code: city_name, latitude, longitude, altitude_ft
us_cities_df = pd.read_csv("datasets/original/city/uscities.csv") #data5

==========================================

2.1 Clean `datasets/original/crash` (unfinished data1)

clean data2

In [4]:
xml_root = etree.getroot()

interest_columns = ["EventId","EventDate","Location","Country","Latitude","Longitude","AirportCode","InjurySeverity","AircraftDamage",
                    "AircraftCategory","NumberOfEngines","EngineType","Schedule","TotalUninjured","TotalMinorInjuries",
                    "TotalSeriousInjuries","TotalFatalInjuries","WeatherCondition","BroadPhaseOfFlight","RegistrationNumber","PurposeOfFlight"]

NTSB_crash_df = pd.DataFrame(columns=interest_columns) # NTSB_crash_df: dataframe which collects airport information. -- YD 02/28/2020

In [5]:
for elem in xml_root: # This loop will run only once
    for row in elem: 
        if not(row.attrib["PurposeOfFlight"]=="Business"):  # Only care about business flights whose engine type is not 'Reciprocating'-- YD 02/28/2020
            continue
        if row.attrib["EngineType"]=="Reciprocating": 
            continue
        information = list()
        for interest in interest_columns:
            if not (row.attrib[interest]==""):
                information.append(row.attrib[interest])
            else:
                information.append(np.nan)
        row_information = pd.Series(information,index=interest_columns)
        NTSB_crash_df = NTSB_crash_df.append(row_information,ignore_index=True)
            
# this may need to run for a while. It takes 18 seconds on my computer
#     NTSB_crash_df

In [6]:
# NTSB_crash_df[(NTSB_crash_df["AircraftCategory"]=="Airplane") & (NTSB_crash_df["EngineType"]!="Reciprocating") & (NTSB_crash_df["Country"]=="United States")]

In [7]:
# (unfinished Crash Data Cleaning.ipynb)

==========================================

2.2 Clean `datasets/original/delay/2009.csv`. Export `unpleasant_2009.csv`

clean data3

In [ ]:
# DEPARTURE
unpleasant_2009_departure = pd.DataFrame()
unpleasant_2009_departure['total_departure'] = delay_2009_df.loc[:,["ORIGIN"]].groupby('ORIGIN').size()
unpleasant_2009_departure[["average_departure_delay","average_departure_taxi"]] = delay_2009_df.loc[:,["ORIGIN","DEP_DELAY","TAXI_OUT"]].groupby('ORIGIN').mean()
unpleasant_2009_departure['average_departure_cancelled'] = delay_2009_df.loc[:,["ORIGIN","CANCELLED"]].groupby('ORIGIN').mean()
unpleasant_2009_departure['averge_departure_distance'] = delay_2009_df.loc[:,["ORIGIN","DISTANCE"]].groupby('ORIGIN').mean()
    
#ARRIVAL
unpleasant_2009_arrival = pd.DataFrame()
unpleasant_2009_arrival['total_arrival'] = delay_2009_df.loc[:,["DEST"]].groupby('DEST').size()
unpleasant_2009_arrival[["average_arrival_delay","average_arrival_taxi"]] = delay_2009_df.loc[:,["DEST","ARR_DELAY","TAXI_IN"]].groupby('DEST').mean()
unpleasant_2009_arrival['average_arrival_diverted'] = delay_2009_df.loc[:,["DEST","DIVERTED"]].groupby('DEST').mean()
unpleasant_2009_arrival['averge_arrival_distance'] = delay_2009_df.loc[:,["DEST","DISTANCE"]].groupby('DEST').mean()
    
unpleasant_2009_departure['total_departure_lg10'] = unpleasant_2009_departure['total_departure'].apply(np.log10)
unpleasant_2009_arrival['total_arrival_lg10'] = unpleasant_2009_arrival['total_arrival'].apply(np.log10)

unpleasant_2009 = unpleasant_2009_departure.merge(unpleasant_2009_arrival,left_index=True,right_index=True)
unpleasant_2009.index.names = ["airport_code"]

In [ ]:
unpleasant_2009

export `unpleasant_2009.csv`

In [ ]:
unpleasant_2009.to_csv("datasets/cleaned/unpleasant_2009.csv")

==========================================

2.3 Clean `datasets/original/airport/airport-extended.csv`. Generate `airport_loc_df` with column names city_name, latitude, longitude, altitude_ft.

In [ ]:
#clean city_name
def clean_city_name(input_city):
    original = input_city
    input_city = str(input_city)
    input_city = input_city.strip()
    input_city = input_city.lower()
    
    input_city = input_city.replace(".","")
    input_city = input_city.replace("\\\\","")
    input_city = input_city.replace("-"," ")
    input_city = input_city.replace(" - "," ")
    input_city = input_city.replace("saint ","st")
    input_city = input_city.replace("east ","")
    input_city = input_city.replace("west ","")
    
    input_city = input_city.translate({ord(k): None for k in digits})
    
    if ('/' in input_city):
        input_city = input_city[:input_city.find('/')]
    if ('(' in input_city):
        input_city = input_city[:input_city.find('(')]
    if (',' in input_city):
        input_city = input_city[:input_city.find(',')]
    input_city = input_city.strip()   
    if (' ' in input_city):
        temp=input_city.find(' ')
        if (temp > 2):
            input_city = input_city[:input_city.find(' ')]
        else:
            if (input_city.find(' ',temp+1) != -1):
                input_city = input_city[temp+1:input_city.find(' ',temp+1)]
            else:
                input_city = input_city[temp+1:]
    input_city = input_city.strip()
    try:
        assert len(input_city) > 2
        assert input_city.replace(" ","").replace("'","").isalpha()
    except:
        print("This city name is prehaps incorrect: ",original,input_city,len(original))
    return input_city

In [ ]:
airport_loc_df.columns = ["ID","name","city_name","country","airport_code","code4","latitude","longitude","altitude_ft","UTC_offset","DST","timezone","type","information_source"]
airport_loc_df = airport_loc_df.loc[:,["city_name","country","airport_code","latitude","longitude","altitude_ft"]]
airport_loc_df = airport_loc_df[airport_loc_df["country"]=="United States"]
airport_loc_df = airport_loc_df.loc[:,["city_name","airport_code","latitude","longitude","altitude_ft"]]
airport_loc_df = airport_loc_df[airport_loc_df["airport_code"]!="\\N"] # remove NAN in index
airport_loc_df = airport_loc_df.set_index("airport_code")

airport_loc_df["city_name"] = airport_loc_df["city_name"].apply(clean_city_name)

In [ ]:
airport_loc_df

In [ ]:
#airport_loc_df.to_csv("datasets/cleaned/airport_loc.csv")

==========================================

2.4 Clean `datasets/original/city/uscities.csv`.

clean data5

In [ ]:
us_cities_df = us_cities_df[["city","state_id","county_fips","county_name","population","density","lat","lng"]]
us_cities_df = us_cities_df.rename(columns = {"city":"city_name"})
us_cities_df["fips"] = us_cities_df["county_fips"]

def get_county_code(input_county):
    return int(input_county) % 1000

us_cities_df["county_fips"] = us_cities_df["county_fips"].apply(get_county_code)

us_cities_df["city_name"] = us_cities_df["city_name"].apply(clean_city_name)

2.5 Merge `unpleasant_2009` and `airport_loc_df` to get pre-`airport_prop_df`. Drop rows whose city_name isn't included in `us_cities_df`. Scrape pcp data with datasets in `datasets/original/weather`. Then we get `airport_prop_df`. Export airport_prop_df.csv`.

In [ ]:
airport_prop_df = unpleasant_2009.merge(airport_loc_df,how='inner',left_index=True,right_index=True)

In [ ]:
#airport_code_df = airport_prop_df[[]]
#airport_code_df
#airport_code_df.to_csv("datasets/cleaned/airtport_code.csv")

In [ ]:
airport_prop_df

In [ ]:
airport_prop_df = airport_prop_df[["city_name", "latitude", "longitude", "altitude_ft" ]]

In [ ]:
airport_prop_df

In [ ]:
city_climate_df = pd.DataFrame(columns=["airport_code","population","density","avg_temp_sp","avg_temp_su","avg_temp_fa","avg_temp_wi","avg_precipitation_sp","avg_precipitation_su","avg_precipitation_fa","avg_precipitation_wi"])
city_search_df = pd.DataFrame(columns=["airport_code","state_id","county_id","city_id","fips"])

In [ ]:
for ind,row in airport_prop_df.iterrows():
    city = row["city_name"]
    target_lat = row["latitude"]
    target_lng = row["longitude"]
    try:      
        target_cities = us_cities_df[us_cities_df["city_name"]==city]
        
        if not (target_cities.shape[0] == 1):
            def calc_dis(input_):
                err = abs(target_lat - input_["lat"]) + abs(target_lng - input_["lng"])
                return err
            target_cities.loc[:,"error"] = (target_cities.apply(calc_dis,axis=1))
            target_city = target_cities.sort_values(by="error").iloc[0]
            
            assert target_city["error"] < 1.5
            
            target_city = target_city.drop(["error"])
        elif (target_cities.shape[0] >= 1):
            target_city = target_cities.iloc[0]
        
        county = str(target_city["county_fips"])
        if (len(county)==1):
            county = "00" + county
        elif (len(county)==2):
            county = "0" + county

        city_search_df = city_search_df.append({"airport_code":ind,"state_id":target_city["state_id"],"county_id":county,"city_id":target_city.name,"fips":target_city["fips"]},ignore_index=True)        
    except:
        try:
            def calc_dis(input_):
                err = abs(target_lat - input_["lat"]) + abs(target_lng - input_["lng"])
                return err
            us_cities_df_copy = us_cities_df
            us_cities_df_copy.loc[:,"error"] = (us_cities_df.apply(calc_dis,axis=1))
            target_city = us_cities_df_copy.sort_values(by="error").iloc[0]
            assert target_city["error"] < 1.5
            county = str(target_city["county_fips"])
            if (len(county)==1):
                county = "00" + county
            elif (len(county)==2):
                county = "0" + county
            city_search_df = city_search_df.append({"airport_code":ind,"state_id":target_city["state_id"],"county_id":county,"city_id":target_city.name,"fips":target_city["fips"]},ignore_index=True)
        except:
            print("No data for ",city)
            city_search_df = city_search_df.append({"airport_code":ind,"state_id":np.nan,"county_id":np.nan,"city_id":np.nan,"fips":np.nan},ignore_index=True)

#special case for DC
for ind,row in city_search_df.iterrows():
    if (row["airport_code"]=="DCA"):
        city_search_df.iloc[ind]["state_id"]="MD"
        city_search_df.iloc[ind]["county_id"]="511"

        
        
city_search_df = city_search_df[(city_search_df["state_id"]!="HI") & (city_search_df["state_id"]!="AK")]

In [ ]:
city_search_df.head()

scrape cities in `city_search_df`

In [ ]:
def download_climate_data(state,county,year):
    save_path = "datasets/original/weather/"
    fname = state + county + "_" + str(year) + ".csv"
    if (len(glob.glob(save_path + fname))==0):
        URL = "https://www.ncdc.noaa.gov/cag/county/time-series/{}-{}-{}-all-1-2000-2020.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000".format(state,county,"tavg")
        r = requests.get(URL)
        file = wget.download(URL,out=save_path + "tavg/tavg_" + fname)
        URL = "https://www.ncdc.noaa.gov/cag/county/time-series/{}-{}-{}-all-1-2000-2020.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000".format(state,county,"pcp")
        r = requests.get(URL)
        file = wget.download(URL,out=save_path + "pcp/pcp_" + fname)

        tavg_df = pd.read_csv(save_path + "tavg/tavg_" + fname).iloc[4:]
        tavg_df.columns=["date","tavg","comp"]
        tavg = tavg_df.set_index("date")["tavg"]

        pcp_df = pd.read_csv(save_path + "pcp/pcp_" + fname).iloc[4:]
        pcp_df.columns=["date","pcp","comp"]
        pcp = pcp_df.set_index("date")["pcp"]

        pd.concat([tavg, pcp], axis=1).to_csv(save_path + fname)
        time.sleep(1) # not requesting too frequently

In [ ]:
counter = 0
for ind,row in city_search_df.iterrows():
    try:
        download_climate_data(row["state_id"],row["county_id"],2018)
        download_climate_data(row["state_id"],row["county_id"],2019)
    except:
        print(row)
    counter+=1
    print("progress: {:.2f}%   Just done: {}".format(100 * counter / city_search_df.shape[0],row["airport_code"]),end="\r")


temp_pcp_df = pd.DataFrame(columns=["airport_code","temp_avg","pcp_avg"])
for ind,row in city_search_df.iterrows():
    state = row["state_id"]
    county = row["county_id"]
    save_path = "datasets/original/weather/"
    years = [2018,2019]
    
    try:
        tavg = 0
        pcp = 0
        for year in years:
            fname = state + county + "_" + str(year) + ".csv"
            temp_pcp = pd.read_csv(save_path + fname)
            tavg += temp_pcp.mean()["tavg"]
            pcp += temp_pcp.mean()["pcp"]
        tavg /= len(years)
        pcp /= len(years)
        
        temp_pcp_df = temp_pcp_df.append({"airport_code":row["airport_code"],"temp_avg":tavg,"pcp_avg":pcp},ignore_index=True)
        
    except:
        temp_pcp_df = temp_pcp_df.append({"airport_code":row["airport_code"],"temp_avg":np.nan,"pcp_avg":np.nan},ignore_index=True)

In [ ]:
airport_prop_df = airport_prop_df.merge(city_search_df.set_index("airport_code").loc[:,["city_id","fips"]],left_index=True,right_index=True)
airport_prop_df = airport_prop_df.merge(temp_pcp_df.set_index("airport_code"),left_index=True,right_index=True)

In [ ]:
airport_prop_df

In [ ]:
airport_prop_df.to_csv("datasets/cleaned/airport_prop.csv")

2.6 Clean `datasets/original/airports/Bird Strikes.xlsx`. (unfinished)